<a href="https://colab.research.google.com/github/bhujithk/mlCodeSnippets/blob/master/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving review_test.csv to review_test.csv
Saving review_train.csv to review_train.csv


In [3]:
!pip install contractions
!pip install textsearch
!pip install tqdm

     |████████████████████████████████| 245kB 6.8MB/s 
     |████████████████████████████████| 317kB 43.9MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81706 sha256=be51633113aea301bffc6eb86f7c8dc8a6d61b1b838c7c89cbe27538852b3792
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [4]:
import nltk
nltk.download('punkt')
import pandas as pd

dataset = pd.read_csv(r'review_train.csv')
dataset.info()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14825 entries, 0 to 14824
Data columns (total 3 columns):
Text         14825 non-null object
Score        14825 non-null int64
Sentiment    14825 non-null int64
dtypes: int64(2), object(1)
memory usage: 347.5+ KB


In [5]:
dataset.head()

,Text,Score,Sentiment
0,"I got a free sample of these once, and now--we...",5,1
1,I used to get this Tea when I lived in Washing...,4,1
2,This is my all time favorite 'grab and go' sna...,5,1
3,This flavor is very good and unexpected. The ...,4,1
4,thrilled to have this assortment as i got the ...,4,1


In [6]:
dataset_test = pd.read_csv(r'review_test.csv')
dataset_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3707 entries, 0 to 3706
Data columns (total 3 columns):
Text         3707 non-null object
Score        3707 non-null int64
Sentiment    3707 non-null int64
dtypes: int64(2), object(1)
memory usage: 87.0+ KB


In [0]:
train_reviews = dataset['Text'].values
train_sentiments = dataset['Sentiment'].values

test_reviews = dataset_test['Text'].values 
test_sentiments = dataset_test['Sentiment'].values 

In [0]:
import contractions
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata


def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    norm_docs.append(doc)
  return norm_docs

In [9]:
%%time

norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

100%|██████████| 3707/3707 [00:00<00:00, 4548.44it/s]

CPU times: user 3.96 s, sys: 110 ms, total: 4.07 s
Wall time: 4.12 s


In [10]:
%%time

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# build BOW features on train reviews
cv = CountVectorizer(binary=False, min_df=5, max_df=1.0, ngram_range=(1,2))
cv_train_features = cv.fit_transform(norm_train_reviews)


# build TFIDF features on train reviews
tv = TfidfVectorizer(use_idf=True, min_df=5, max_df=1.0, ngram_range=(1,2),
                     sublinear_tf=True)
tv_train_features = tv.fit_transform(norm_train_reviews)

CPU times: user 6.4 s, sys: 208 ms, total: 6.61 s
Wall time: 6.64 s


In [11]:
%%time

# transform test reviews into features
cv_test_features = cv.transform(norm_test_reviews)
tv_test_features = tv.transform(norm_test_reviews)

CPU times: user 1.05 s, sys: 6.72 ms, total: 1.06 s
Wall time: 1.07 s


In [12]:
print('BOW model:> Train features shape:', cv_train_features.shape, ' Test features shape:', cv_test_features.shape)
print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

BOW model:> Train features shape: (14825, 36702)  Test features shape: (3707, 36702)
TFIDF model:> Train features shape: (14825, 36702)  Test features shape: (3707, 36702)


In [13]:
%%time

# Logistic Regression model on BOW features
from sklearn.linear_model import LogisticRegression

# instantiate model
lr = LogisticRegression(penalty='l2', max_iter=500, C=1, solver='lbfgs', random_state=42)

# train model
lr.fit(cv_train_features, train_sentiments)

# predict on test data
lr_bow_predictions = lr.predict(cv_test_features)

CPU times: user 4.02 s, sys: 2.61 s, total: 6.63 s
Wall time: 3.4 s


In [14]:

from sklearn.metrics import confusion_matrix, classification_report

labels = ['negative', 'positive']
print(classification_report(test_sentiments, lr_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, lr_bow_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.84      0.68      0.75       592
           1       0.94      0.97      0.96      3115

    accuracy                           0.93      3707
   macro avg       0.89      0.83      0.85      3707
weighted avg       0.92      0.93      0.92      3707



,negative,positive
negative,404,188
positive,79,3036


In [15]:

%%time

# Logistic Regression model on TF-IDF features

# train model
lr.fit(tv_train_features, train_sentiments)

# predict on test data
lr_tfidf_predictions = lr.predict(tv_test_features)

CPU times: user 907 ms, sys: 603 ms, total: 1.51 s
Wall time: 784 ms


In [16]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, lr_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, lr_tfidf_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.95      0.41      0.57       592
           1       0.90      1.00      0.94      3115

    accuracy                           0.90      3707
   macro avg       0.93      0.70      0.76      3707
weighted avg       0.91      0.90      0.89      3707



,negative,positive
negative,242,350
positive,12,3103


In [17]:
%%time

# Random Forest model on BOW features
from sklearn.ensemble import RandomForestClassifier

# instantiate model
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)

# train model
rf.fit(cv_train_features, train_sentiments)

# predict on test data
rf_bow_predictions = rf.predict(cv_test_features)

CPU times: user 24.3 s, sys: 45.6 ms, total: 24.3 s
Wall time: 12.6 s


In [18]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, rf_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, rf_bow_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.99      0.17      0.29       592
           1       0.86      1.00      0.93      3115

    accuracy                           0.87      3707
   macro avg       0.93      0.59      0.61      3707
weighted avg       0.88      0.87      0.83      3707



,negative,positive
negative,102,490
positive,1,3114


In [19]:
%%time

# Random Forest model on TF-IDF features

# train model
rf.fit(tv_train_features, train_sentiments)

# predict on test data
rf_tfidf_predictions = rf.predict(tv_test_features)

CPU times: user 25.9 s, sys: 33.8 ms, total: 25.9 s
Wall time: 13.2 s


In [20]:

labels = ['negative', 'positive']
print(classification_report(test_sentiments, rf_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, rf_tfidf_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.97      0.20      0.33       592
           1       0.87      1.00      0.93      3115

    accuracy                           0.87      3707
   macro avg       0.92      0.60      0.63      3707
weighted avg       0.88      0.87      0.83      3707



,negative,positive
negative,119,473
positive,4,3111


In [0]:
import gensim
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Dense
from sklearn.preprocessing import LabelEncoder

In [0]:
le = LabelEncoder()
# tokenize train reviews & encode train labels
tokenized_train = [nltk.word_tokenize(text)
                       for text in norm_train_reviews]
y_train = le.fit_transform(train_sentiments)
# tokenize test reviews & encode test labels
tokenized_test = [nltk.word_tokenize(text)
                       for text in norm_test_reviews]
y_test = le.fit_transform(test_sentiments)

In [24]:
# print class label encoding map and encoded labels
print('Sentiment class label map:', dict(zip(le.classes_, le.transform(le.classes_))))
print('Sample test label transformation:\n'+'-'*35,
      '\nActual Labels:', test_sentiments[:3], '\nEncoded Labels:', y_test[:3])

Sentiment class label map: {0: 0, 1: 1}
Sample test label transformation:
----------------------------------- 
Actual Labels: [1 1 1] 
Encoded Labels: [1 1 1]


In [0]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [26]:
%%time
# build word2vec model
w2v_num_features = 300
w2v_model = gensim.models.Word2Vec(tokenized_train, size=w2v_num_features, window=150,
                                   min_count=10, workers=4, iter=5)

2019-09-18 07:06:54,419 : INFO : collecting all words and their counts
2019-09-18 07:06:54,420 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-09-18 07:06:54,568 : INFO : PROGRESS: at sentence #10000, processed 786775 words, keeping 29049 word types
2019-09-18 07:06:54,639 : INFO : collected 36775 word types from a corpus of 1167142 raw words and 14825 sentences
2019-09-18 07:06:54,640 : INFO : Loading a fresh vocabulary
2019-09-18 07:06:54,875 : INFO : effective_min_count=10 retains 5110 unique words (13% of original 36775, drops 31665)
2019-09-18 07:06:54,876 : INFO : effective_min_count=10 leaves 1107625 word corpus (94% of original 1167142, drops 59517)
2019-09-18 07:06:54,900 : INFO : deleting the raw counts dictionary of 36775 items
2019-09-18 07:06:54,903 : INFO : sample=0.001 downsamples 57 most-common words
2019-09-18 07:06:54,904 : INFO : downsampling leaves estimated 790586 word corpus (71.4% of prior 1107625)
2019-09-18 07:06:54,919 : INFO : 

CPU times: user 1min, sys: 110 ms, total: 1min
Wall time: 31.2 s


In [0]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [0]:
avg_wv_train_features = averaged_word2vec_vectorizer(corpus=tokenized_train, model=w2v_model,
                                                     num_features=w2v_num_features)
avg_wv_test_features = averaged_word2vec_vectorizer(corpus=tokenized_test, model=w2v_model,
                                                    num_features=w2v_num_features)

In [29]:

print('Word2Vec model:> Train features shape:', avg_wv_train_features.shape, ' Test features shape:', avg_wv_test_features.shape)

Word2Vec model:> Train features shape: (14825, 300)  Test features shape: (3707, 300)


In [0]:
def construct_deepnn_architecture(num_input_features):
    dnn_model = Sequential()
    dnn_model.add(Dense(512, input_shape=(num_input_features,)))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(256))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(256))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(1))
    dnn_model.add(Activation('sigmoid'))

    dnn_model.compile(loss='binary_crossentropy', optimizer='adam',                 
                      metrics=['accuracy'])
    return dnn_model

In [31]:
w2v_dnn = construct_deepnn_architecture(num_input_features=w2v_num_features)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2019-09-18 07:09:10,220 : WARNING : From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2019-09-18 07:09:10,391 : WARNING : From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [32]:

w2v_dnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               154112    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               6

In [33]:
batch_size = 100
w2v_dnn.fit(avg_wv_train_features, y_train, epochs=10, batch_size=batch_size, 
            shuffle=True, validation_split=0.1, verbose=1)

Train on 13342 samples, validate on 1483 samples
Epoch 1/10
13342/13342 [==============================] - 2s 124us/sample - loss: 0.3232 - acc: 0.8620 - val_loss: 0.2985 - val_acc: 0.8759
Epoch 2/10
13342/13342 [==============================] - 1s 90us/sample - loss: 0.2944 - acc: 0.8728 - val_loss: 0.2971 - val_acc: 0.8726
Epoch 3/10
13342/13342 [==============================] - 1s 88us/sample - loss: 0.2858 - acc: 0.8763 - val_loss: 0.2940 - val_acc: 0.8800
Epoch 4/10
13342/13342 [==============================] - 1s 90us/sample - loss: 0.2831 - acc: 0.8790 - val_loss: 0.2890 - val_acc: 0.8786
Epoch 5/10
13342/13342 [==============================] - 1s 89us/sample - loss: 0.2756 - acc: 0.8814 - val_loss: 0.2873 - val_acc: 0.8759
Epoch 6/10
13342/13342 [==============================] - 1s 92us/sample - loss: 0.2678 - acc: 0.8834 - val_loss: 0.2829 - val_acc: 0.8813
Epoch 7/10
13342/13342 [==============================] - 1s 87us/sample - loss: 0.2660 - acc: 0.8839 - val_loss: 0.

In [34]:
y_pred = w2v_dnn.predict_classes(avg_wv_test_features)
predictions = le.inverse_transform(y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [35]:

labels = ['negative', 'positive']
print(classification_report(test_sentiments, predictions))
pd.DataFrame(confusion_matrix(test_sentiments, predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.70      0.41      0.52       592
           1       0.90      0.97      0.93      3115

    accuracy                           0.88      3707
   macro avg       0.80      0.69      0.73      3707
weighted avg       0.87      0.88      0.87      3707



,negative,positive
negative,245,347
positive,103,3012
